In [1]:
import geopandas as gpd
from geopandas.geoseries import *
import pandas as pd 

# Get shapes of Berlin's Planungsräume 
# 448 PLRs
# source: Amt für Statistik Berlin-Brandenburg
# https://fbinter.stadt-berlin.de/fb/berlin/service_intern.jsp?id=s_lor_plan@senstadt&type=WFS

file_path_of_plr = 'LOR_Planungsraeume_2019.geojson'
plr_geo = gpd.read_file(file_path_of_plr)

# add new id and save file
plr_geo.insert(0, 'New_ID', range(0, 0 + len(plr_geo)))
plr_geo.to_file('output_berlin-lor.planungsraeume.geojson', driver='GeoJSON')

# Get additional data to Berlin's Planungsräume
# source see above (FIS-Broker)

plr_data = pd.read_csv('LOR_Planungsraeume_2019.csv', sep=';')
plr_data.rename(columns={'Planungsraum': 'PLANUNGSRAUM', 'Bezirksregion': 'BEZIRKSREGION'}, inplace=True)

plr = plr_geo.merge(plr_data, on=['PLANUNGSRAUM', 'BEZIRKSREGION'])
plr.drop(['DATUM_GUELTIG_AB', 'Bezirk', 'Prognoseraum', 'Flächengröße in m²'], axis=1, inplace=True)
plr.rename(columns={'Schlüssel': 'SCHLUESSEL'}, inplace=True)
plr['SCHLUESSEL'] = plr['SCHLUESSEL'].astype(str)

# buffer and projection in meter
projection = "EPSG:5678" # https://epsg.io/5678
# buffer in meter (3 km/h walking pace; what is reachable in 10 min if your start at the boundary of PLR) => 500 m
bufferSize = 500 
plr = plr.to_crs(projection)

In [2]:
# 11522 forests
file_path_of_forests = 'waldflaechen.geojson'
forests = gpd.read_file(file_path_of_forests)
forests.insert(0, 'New_ID', range(0, 0 + len(forests)))
forests.to_file("output_waldflaechen.geojson", driver="GeoJSON")
#forests = forests.to_crs(projection)
forests["area_text"] = (forests["area_text"].replace(',', '.', regex=True).astype(float))

# # 433585749.9072622 => 43358.5749907 hectar Berlin and Brandenburg
# # forests['area_text'].sum()
# # 282036587.22155315 => 28203.658722155315 hectar in Berlin
# # forests[(forests.land == 'Berlin')]['area_text'].sum()
# # forests[(forests.land == 'Brandenburg')]['area_text'].sum() #151549162.68570915

plr['area'] = None
plr['bufferedGeometry'] = None
plr['bufferedArea'] = None
plr['bufferedArea'] = None

bufferedGeometry = plr.geometry.to_crs(projection)
copyGeometry = plr.geometry.to_crs(projection)
bufferedGeometry = bufferedGeometry.buffer(bufferSize)

for index, row in plr.iterrows():
    plr.loc[index, 'area'] = copyGeometry[index].area / 10000
    plr.loc[index, 'bufferedArea'] = bufferedGeometry[index].area / 10000

copyGeometry = bufferedGeometry.to_crs("EPSG:4326")
plr['bufferedGeometry'] = bufferedGeometry
    
# add open space to calc which of them intersects with bufferedGeometry 
file_path_of_open_spaces = 'gruenflaechen_shapes.geojson'
open_spaces = gpd.read_file(file_path_of_open_spaces)
open_spaces.KATASTERFL.sum()
open_spaces['AREA'] = None
open_spaces = open_spaces.to_crs(projection)

for index, row in open_spaces.iterrows():
    open_spaces.loc[index, 'AREA'] = row.geometry.area
open_spaces.AREA.sum() / 10000
open_spaces.insert(0, 'New_ID', range(len(forests), len(forests) + len(open_spaces)))
open_spaces.to_file('output_gruenflaechen_shapes.geojson', driver='GeoJSON')

#check TODO
data = pd.read_csv('data_PLZ.csv', sep=';')
data = data.drop(columns="geometry")
data = data.drop(columns="bufferedGeometry")
data = data[['New_ID', 'SCHLUESSEL', 'PLR_NAME']]
data['New_ID'] = data['New_ID'].astype(str)
data['SCHLUESSEL'] = data['SCHLUESSEL'].astype(str)

gdf = gpd.GeoDataFrame(data, geometry = gpd.GeoSeries(plr['bufferedGeometry']))
gdf.to_file('buffer.geojson', driver='GeoJSON')


# 2509 open spaces
open_spaces = open_spaces.to_crs(projection)

plr['openSpacesInBufferedPLR'] = np.empty((len(plr), 0)).tolist()
plr['openSpacesInPLR'] = np.empty((len(plr), 0)).tolist()
plr['allOpenSpacesArea'] = np.empty((len(plr), 0)).tolist()
plr['allOpenSpacesBufferedArea'] = np.empty((len(plr), 0)).tolist()

# print(open_spaces['KATASTERFL'].sum()) # => 61376300.8 => 6137.63008 hectar

for indexPLR, rowPLR in plr.iterrows():
    for indexOS, rowOS in open_spaces.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(str(rowOS.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesBufferedArea'].append(rowOS.KATASTERFL)
        if rowPLR.geometry.intersects(rowOS.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(str(rowOS.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesArea'].append(rowOS.KATASTERFL)
        if rowPLR.SCHLUESSEL.startswith('0'):
            plr.loc[indexPLR, 'SCHLUESSEL'] = rowPLR.SCHLUESSEL[1:]
    for indexF, rowF in forests.iterrows():
        if rowPLR.bufferedGeometry.intersects(rowF.geometry):
            plr.loc[indexPLR, 'openSpacesInBufferedPLR'].append(str(rowF.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesBufferedArea'].append(rowF.area_text)
        if rowPLR.geometry.intersects(rowF.geometry):
            plr.loc[indexPLR, 'openSpacesInPLR'].append(str(rowF.New_ID))
            plr.loc[indexPLR, 'allOpenSpacesArea'].append(rowF.area_text)

plr['bufferedGeometry'] = copyGeometry
newplr = plr

# sum of all PLRs => 89132.31009939541 hectar
# sumOS = plr['area'].sum()
# print(sumOS)

newplr.astype({'geometry': str}, {'bufferedGeometry': str}).to_csv('PLR_buffered.csv', sep=';')

In [3]:
import pandas as pd 
import numpy as np


newplr['numberOfSpacesInPLR'] = None
newplr['numberOfSpacesInBufferedPLR'] = None

for index, row in newplr.iterrows():
    newplr.loc[index, 'numberOfSpacesInPLR'] = len(row.openSpacesInPLR)
    newplr.loc[index, 'numberOfSpacesInBufferedPLR'] = len(row.openSpacesInBufferedPLR)

data = pd.read_csv('EWR201812E_Matrix.csv', sep=';')
data.rename(columns = {'RAUMID':'SCHLUESSEL'}, inplace = True)
data = data[['SCHLUESSEL', 'E_E']]

data['SCHLUESSEL'] = data['SCHLUESSEL'].astype(str)
newplr['SCHLUESSEL'] = newplr['SCHLUESSEL'].astype(str)

output = pd.merge(newplr, data, on='SCHLUESSEL')
output = output.rename(columns={'E_E': 'population'})

for index, row in output.iterrows():
    output.loc[index, 'allOpenSpacesBufferedArea'] = pd.Series(row.allOpenSpacesBufferedArea).sum() / 10000
    output.loc[index, 'allOpenSpacesArea'] = pd.Series(row.allOpenSpacesArea).sum() / 10000

# 3748148
# print(output['population'].sum())

output = output.to_crs('EPSG:4326')
#del output['bufferedGeometry']
output.astype({'geometry': str}).to_csv('data.csv', index=False, header=True)
output.head()


,New_ID,BEZIRKSNAME,PLANUNGSRAUM,BEZIRKSREGION,PROGNOSERAUM,FLAECHENGROESSE_IN_M2,geometry,SCHLUESSEL,area,bufferedGeometry,bufferedArea,openSpacesInBufferedPLR,openSpacesInPLR,allOpenSpacesArea,allOpenSpacesBufferedArea,numberOfSpacesInPLR,numberOfSpacesInBufferedPLR,population
0,0,Mitte,Nördlicher Landwehrkanal,Tiergarten Süd,Zentrum,934609.72,"POLYGON ((13.35159 52.50976, 13.35180 52.50944...",1011105,93.5224,"POLYGON ((13.34463 52.50831, 13.34450 52.50855...",421.314,"[11804, 11805, 11806, 11808, 11811, 11812, 118...","[11804, 13011, 13017, 13200, 13202, 13205, 132...",172.142,192.299,11,29,1421
1,1,Mitte,Wilhelmstraße,Regierungsviertel,Zentrum,634326.80,"POLYGON ((13.37678 52.51255, 13.37681 52.51246...",1011201,63.48,"POLYGON ((13.36920 52.51128, 13.36921 52.51134...",312.579,"[11596, 11599, 11804, 11806, 11812, 11815, 118...","[13017, 13214]",167,189.775,2,28,2405
2,2,Mitte,Unter den Linden Nord,Regierungsviertel,Zentrum,744742.10,"POLYGON ((13.37673 52.51603, 13.37786 52.51611...",1011202,74.5257,"POLYGON ((13.36953 52.51751, 13.36948 52.51800...",364.318,"[12884, 12887, 12888, 12889, 12891, 12897, 128...","[12884, 13015, 13019, 13022, 13024, 13026, 130...",4.65533,205.364,8,34,632
3,3,Mitte,Unter den Linden Süd,Regierungsviertel,Zentrum,548158.88,"POLYGON ((13.38109 52.51622, 13.38160 52.51526...",1011203,54.8328,"POLYGON ((13.37858 52.51100, 13.37831 52.51108...",324.835,"[11614, 12884, 12887, 12888, 12889, 12890, 128...","[13028, 13036]",0.4778,193.612,2,32,1115
4,4,Mitte,Leipziger Straße,Regierungsviertel,Zentrum,763845.47,"POLYGON ((13.38679 52.51184, 13.38690 52.51115...",1011204,76.4631,"POLYGON ((13.38555 52.50428, 13.38554 52.50429...",362.886,"[11585, 11589, 11595, 11596, 11597, 11599, 116...","[11614, 12887, 12893, 12900, 12901, 13033, 13037]",1.86165,25.989,7,38,8279


In [4]:
import geopandas as gpd
from geopandas.geoseries import *

# source: https://fbinter.stadt-berlin.de/fb/index.jsp?loginkey=zoomStart&mapId=gris_oeffgruen@senstadt&bbox=387456,5818130,395835,5822518
data = pd.read_csv("gruenflaechen.csv").replace('-', '', regex=True)
data.head()
#data["Größe in m² (Kataster)"] = pd.to_numeric(data["Größe in m² (Kataster)"], downcast="float")
#data["Größe in m² (Kataster)"].sum() / 10000
# 6137.6304 ha <-> 6144.21496

#data.head()

,technischer Schlüssel,Objektnummer,Bezirk,Ortsteil,Art der Grünanlage,Name der Grünanlage,Namenszusatz der Grünanlage,Baujahr,letztes Sanierungsjahr,Größe in m² (Kataster),Widmung,Nummer des Planungsraumes,Name Planungsraum,anrechenbar Grünflächenversorgung
0,00008100:00199415,147,MarzahnHellersdorf,Marzahn,Grünanlage,Barnimplatz und Weg entlang Straßenbahn,Havemannstraße und Wittenberger Straße,1994,,9660.2,gewidmet,10010102,Havemannstr.,1
1,00008100:00199478,2300175,MarzahnHellersdorf,Hellersdorf,Grünanlage,Grünfläche (DEST),Oschatzer Ring,,,1048,gewidmet,10020416,Böhlener Str.,1
2,00008100:0019947a,2301127,MarzahnHellersdorf,Hellersdorf,Grünanlage,Kurt Julius Goldstein Park,Riesaer/ Heidenauer Straße,,,35327,gewidmet,10020517,AdeleSandrockStr.,1
3,00008100:00199457,204,MarzahnHellersdorf,Marzahn,Grünanlage,Landsberger Allee 565,Dreiecksfläche Ringenwalder/Brodowiner R,1996,,5802,gewidmet,10010205,MarzahnOst,1
4,00008100:0019942f,211,MarzahnHellersdorf,Marzahn,Grünanlage,Blumberger Damm/ Wuhletalstraße,Grünanlage mit Teich ggü.Kemberger Str.,1986,,4553.8,gewidmet,10010204,Wuhletalstr.,1


In [5]:
# data = gpd.read_file('test.geojson')
# len(data) #2525
# data = gpd.read_file('gruenflaechen_shapes.geojson')

# len(data) #2509
